In [ ]:
import pandas as pd
from datetime import datetime,date,time,timedelta

In [ ]:
VERSION = "FIXRISK_V3_3_80"

In [1]:
COMMODITY_NAME = "CRUDEOIL"
LOT_SIZE = 100
MAX_DAYS = 10 # The days we want to look back. 
SL_PERCENTILE = 20
SL_CHANGE = 3*60*60 # IN SECONDS (3 HOURS)

MAX_RISK_PER_TRADE = 10**5 # 1 LAC 

SD_LIMIT = 15

MAX_SL = 2000000
MIN_SL = -0.5

MAX_LOTS = 50000 # 8 lots per trade and 24 lots for whole 1 cr position 

STOCH_BUY = [80] # stoch value criteria for buy side
ALLOCATION_BUY = [1] # Equal allocation (50 lacs) for each entry

STOCH_SELL = [20] # same as buy side
ALLOCATION_SELL = [1]

BALANCE = 10**7
MAX_ALLOCATION = 10**7 / len(ALLOCATION_BUY) 
LEVERAGE = 1

RISK_PER_POSITION = int(MAX_RISK_PER_TRADE / len(STOCH_BUY))

COST_PERCENT = 0.05 # PER TRADE (IN PERCENTAGE)

ROLLOVER_DAYS = 7 # Rolling over 7 days prior 
ROLLOVER_TIME = time(12,15,0) # rolling over at 12:15 PM

NO_TRADING = 5 # This variable will remove the first x minutes from every day. Meaning we won't trade for first x minutes of day open.

NameError: name 'time' is not defined

In [ ]:
import os

if(not os.path.exists("./{}".format(VERSION))):
    os.makedirs("./{}".format(VERSION))

if(not os.path.exists("./{}".format('./'+VERSION +"/"+COMMODITY_NAME))):
    os.makedirs("./{}".format('./'+VERSION +"/"+COMMODITY_NAME))

In [ ]:
# df1 is crudeoil 1 data. DF2 is crudeoil 2 and exp is the expiry csv

df1 = pd.read_csv('./'+COMMODITY_NAME+"/"+ "DATA_" + COMMODITY_NAME + "/" + COMMODITY_NAME +"-1.csv")
df2 = pd.read_csv('./'+COMMODITY_NAME+"/"+ "DATA_" + COMMODITY_NAME + "/" + COMMODITY_NAME +"-2.csv")
exp = pd.read_csv('./'+COMMODITY_NAME+"/"+ "DATA_" + COMMODITY_NAME + "/Expiry.csv")
df1['Date_Time'] = df1[['Date', 'Time']].agg(' '.join, axis=1)
df2['Date_Time'] = df2[['Date', 'Time']].agg(' '.join, axis=1)
exp['Expiry']=pd.to_datetime(exp['Expiry'],format=('%Y-%m-%d'))
df1['Date']=pd.to_datetime(df1['Date']).dt.date
df2['Date']=pd.to_datetime(df2['Date']).dt.date

# df1['Date_Time'] = pd.to_datetime(df1['Date'] + df1['Time'], format='%Y-%m-%d%H:%M:%S')
# df2['Date_Time'] = pd.to_datetime(df2['Date'] + df2['Time'], format='%Y-%m-%d%H:%M:%S')

df1.Date_Time = pd.to_datetime(df1.Date_Time)
df2.Date_Time = pd.to_datetime(df2.Date_Time)

In [ ]:
import numpy as np
def add_dte(df,exp):
    #Adding DTE
    if('DTE' in df):
        return df
    
    print("ADDING DTE TO THE DATA...")
    
    exp['Expiry'] = pd.to_datetime(exp['Expiry']).dt.date

    gp = df.groupby('Date')
    

    dflist = []

    for k,res in gp:
        
        dte = None
        DTE = []
        for i in exp['Expiry']:
            
            diff = (i - k).days
            if(diff >= 0):
                dte = diff
                break
        for i in res['Open']:
            DTE.append(diff)
        
        res['DTE'] = DTE
        # res.reset_index(inplace=True)
        # res.set_index('Date_Time',inplace = True)
        # print(res)
        dflist.append(res)

    df = pd.concat(dflist)

    return df

df1=add_dte(df1,exp)
df2=add_dte(df2,exp)
def get_data(df2,date):
    print("QH AND QL UPDATED FROM DF2")
    df = df2[df2.Date < date]
    dates = (df.Date.drop_duplicates()).tolist()
    dates = dates[-10:]
    df = df[df.Date >= dates[0]]
    q_h = []
    q_l = []
    for i in dates:
        q_h.append(df[df.Date == i].Day_High.iloc[0])
        q_l.append(df[df.Date == i].Day_Low.iloc[0])
    # print(df.Date.drop_duplicates())
    return [q_h,q_l]


In [ ]:
def add_dayhilo(df):
    
    print("ADDING DAY_HIGH AND DAY_LOW...")
    
    if('Day_High' in df and 'Day_Low' in df):
        return df
    
    print("ADDING DAY_HIGH AND DAY_LOW...")
    
    gp = df.groupby('Date')

    Hi = []
    Lo = []

    dflist = []

    for k,res in gp:
        day_hi = max(res['High'])
        day_lo = min(res['Low'])
        
        res['Day_High'] = [day_hi]*len(res.Open)
        res['Day_Low'] = [day_lo]*len(res.Open)
        dflist.append(res)
        
    df = pd.concat(dflist)

    return df

df1=add_dayhilo(df1)
df2=add_dayhilo(df2)

In [ ]:
df = df1
days = 10

gp = df.groupby('Date')

q_h = []
q_l = []
dflist = []

prev = 0 

for k,res in gp:
    print(k)
    H_ = []
    L_ = []
    
    h = float('-inf')
    l = float('inf')
    
    if(prev == 1):
        q_h,q_l = get_data(df2,k)
        print(q_h)
        print(q_l)
        prev = 0
    
    for i in q_h:
        if(i > h):
            h = i
    for i in q_l:
        if(i < l):
            l = i
    
    if(len(q_h) == 0):
        h = res.Day_High.iloc[0]
        l = res.Day_Low.iloc[0]
    
    for i in range(len(res['Open'])):
        cur_h = res.High.iloc[i]
        h = max(h,cur_h)
        cur_l = res.Low.iloc[i]
        l = min(l,cur_l)
        H_.append(h)
        L_.append(l)
    
    res['H' + str(days)] = H_
    res['L' + str(days)] = L_
    
    if(len(q_h) == days):
        q_h.pop(0)
        q_h.append(res.Day_High.iloc[0])
        q_l.pop(0)
        q_l.append(res.Day_Low.iloc[0])
    else:
        q_h.append(res.Day_High.iloc[0])
        q_l.append(res.Day_Low.iloc[0])
    dflist.append(res)
    
    if(res.DTE.iloc[0] == 0):
        prev = 1

df = pd.concat(dflist)
df1 = df

In [ ]:
def add_hilo(df,days):
    #Adding H10 and L10

    if(('Day_High' not in df) or ('Day_Low' not in df)):
        # print("ERROR: DAY_HIGH OR DAY_LOW COLUMNS MISSING")
        # return df
        df = add_dayhilo(df)

    req1 = 'H' + str(days)
    req2 = 'L' + str(days)
    
    if((req1 in df) and (req2 in df)):
        return df

    print("ADDING H{} AND L{}".format(days,days))

    if('Date' not in df):
        df.Date = pd.to_datetime(df.Date_Time)
        df.Date = df.Date.dt.date
    
    gp = df.groupby('Date')

    q_h = []
    q_l = []

    dflist = []

    for k,res in gp:
        H_ = []
        L_ = []
        
        h = float('-inf')
        l = float('inf')
        
        for i in q_h:
            if(i > h):
                h = i
        for i in q_l:
            if(i < l):
                l = i
        
        if(len(q_h) == 0):
            h = res.Day_High.iloc[0]
            l = res.Day_Low.iloc[0]
        
        for i in range(len(res['Open'])):
            cur_h = res.High.iloc[i]
            h = max(h,cur_h)
            cur_l = res.Low.iloc[i]
            l = min(l,cur_l)
            H_.append(h)
            L_.append(l)
        
        res['H' + str(days)] = H_
        res['L' + str(days)] = L_
        
        if(len(q_h) == days):
            q_h.pop(0)
            q_h.append(res.Day_High.iloc[0])
            q_l.pop(0)
            q_l.append(res.Day_Low.iloc[0])
        else:
            q_h.append(res.Day_High.iloc[0])
            q_l.append(res.Day_Low.iloc[0])
        dflist.append(res)

    df = pd.concat(dflist)

    return df

df2=add_hilo(df2,MAX_DAYS)

In [ ]:
# ADDING THE TIME FOR H{MAX_DAYS} AND L{MAX_DAYS}
MAX_DAYS = 10
ROLLOVER_DAYS = 7
df = df1
gp = df.groupby('Date')
start_date = None
count = 0
dflist1 = []
prev = 0
last_expiry_date = None
count_days = 0  

for k,res in gp:
    count += 1
    print(k)
    if(count <= MAX_DAYS):
        res['H{}_TIME'.format(MAX_DAYS)] = [None for _ in range(len(res))]
        res['L{}_TIME'.format(MAX_DAYS)] = [None for _ in range(len(res))] 
    else:
        prev_h = res['H{}'.format(MAX_DAYS)].iloc[0]
        prev_l = res['L{}'.format(MAX_DAYS)].iloc[0]
        
        prev_h_time = None
        prev_l_time = None
        
        hdf = df[(df.Date_Time <= res.Date_Time.iloc[0])]
        hdf = hdf[hdf.High == prev_h]
        
        ldf = df[(df.Date_Time <= res.Date_Time.iloc[0])]
        ldf = ldf[ldf.Low == prev_l]
        
        cur_dte = res.DTE.iloc[0]
        
        # if(prev == 1 and count_days < 10):
        #     hdf2 = df2[(df2.Date_Time <= res.Date_Time.iloc[0]) & (hdf2.High == prev_h)]
        #     time1 = datetime(2012,1,1,1,1,1)
        #     time2 = datetime(2012,1,1,1,1,1)
        #     if(len(hdf)):
        #         time1 = hdf.Date_Time.iloc[-1]
        #     else:
        #         None
            
        #     if(len(hdf2)):
        #         time2 = hdf2.Date_Time.iloc[-1]
        #     else:
        #         None
            
        #     if(time1 > time2):
        #         if(len(hdf) > 1):
        #             prev_h_time = hdf.Date_Time.iloc[-1]
        #         else:
        #             prev_h_time = hdf.Date_Time.iloc[0]
        #     else:
        #         if(len(hdf2) >= 1):
        #             prev_h_time = hdf2.Date_Time.iloc[-1]
        #         else:
        #             prev_h_time = hdf2.Date_Time.iloc[0]
        #     count_days += 1
        # else:
        #     prev = 0
        #     if(len(hdf) > 1):
        #         prev_h_time = hdf.Date_Time.iloc[-1]
        #     else:
        #         prev_h_time = hdf.Date_Time.iloc[0]
        
        # if(prev == 1 and count_days < 10):
        #     ldf2 = df2[(df2.Date_Time <= res.Date_Time.iloc[0]) & (ldf2.Low == prev_l)]
        #     time1 = datetime(2012,1,1,1,1,1)
        #     time2 = datetime(2012,1,1,1,1,1)
        #     if(len(ldf)):
        #         time1 = pd.to_datetime(ldf.Date_Time.iloc[-1])
        #     else:
        #         None
            
        #     if(len(ldf2)):
        #         time2 = pd.to_datetime(ldf2.Date_Time.iloc[-1])
        #     else:
        #         None
            
        #     if(time1 > time2):
        #         if(len(ldf) > 1):
        #             prev_l_time = ldf.Date_Time.iloc[-1]
        #         else:
        #             prev_l_time = ldf.Date_Time.iloc[0]
        #     else:
        #         if(len(ldf2) >= 1):
        #             prev_l_time = ldf2.Date_Time.iloc[-1]
        #         else:
        #             prev_l_time = ldf2.Date_Time.iloc[0]
        #     count_days += 1
        # else:
        #     prev = 0
        #     if(len(ldf) > 1):
        #         prev_l_time = ldf.Date_Time.iloc[-1]
        #     else:
                # prev_l_time = ldf.Date_Time.iloc[0]
        
        # if(prev == 1 and count_days < 10):
        #     hdf2 = df2[(df2.Date_Time <= res.Date_Time.iloc[0])]
        #     hdf2 = hdf2[hdf2.High == prev_h]
            
        #     if(len(hdf)):
                
            
        #     if(len(hdf2) >= 1):
        #         prev_h_time = hdf2.Date_Time.iloc[-1]
        #     else:
        #         prev_h_time = hdf2.Date_Time.iloc[0]
        #     # print(prev_h_time)
        #     count_days += 1
        # else: 
        #     prev = 0
        #     count_days = 0
        #     if(len(hdf) > 1):
        #         prev_h_time = hdf.Date_Time.iloc[-1]
        #     elif(len(hdf) == 1):
        #         prev_h_time = hdf.Date_Time.iloc[0]
        #     else:
        #         prev_h_time = res.Date_Time.iloc[0]
            
        # if(prev == 1 and count_days < 10):
            # ldf2 = df2[(df2.Date_Time <= res.Date_Time.iloc[0])]
            # ldf2 = ldf2[ldf2.Low == prev_l]
        #     if(len(ldf2) > 1):
        #         prev_l_time = ldf2.Date_Time.iloc[-1]
        #     elif(len(ldf2) == 1):
        #         prev_l_time = ldf2.Date_Time.iloc[0]
        #     else:
        #         prev_l_time = res.Date_Time.iloc[0]
        #     count_days += 1
        # else:
        #     prev = 0
        #     count_days = 0
        #     if(len(ldf) > 1):
        #         prev_l_time = ldf.Date_Time.iloc[-1]
        #     elif(len(ldf) == 1):
        #         prev_l_time = ldf.Date_Time.iloc[0]
        #     else:
        #         prev_l_time = res.Date_Time.iloc[0]
        
        if((len(hdf) >= 1 and hdf.DTE.iloc[-1] > ROLLOVER_DAYS) or cur_dte <= ROLLOVER_DAYS):
            prev_h_time = hdf.Date_Time.iloc[-1]
        else:
            hdf2 = df2[(df2.Date_Time <= res.Date_Time.iloc[0])]
            hdf2 = hdf2[hdf2.High == prev_h]
            if(len(hdf2) > 1):
                prev_h_time = hdf2.Date_Time.iloc[-1]
            else:
                prev_h_time = hdf2.Date_Time.iloc[0]
            # print(prev_h_time)
        if((len(ldf) >= 1 and ldf.DTE.iloc[-1] > ROLLOVER_DAYS) or cur_dte <= ROLLOVER_DAYS):
            prev_l_time = ldf.Date_Time.iloc[-1]
        else:
            ldf2 = df2[(df2.Date_Time <= res.Date_Time.iloc[0])]
            ldf2 = ldf2[ldf2.Low == prev_l]
            if(len(ldf2) > 1):
                prev_l_time = ldf2.Date_Time.iloc[-1]
            else:
                prev_l_time = ldf2.Date_Time.iloc[0]
            
        if(cur_dte == 0):
            last_expiry_date = k
            prev = 1
        
        
        htime = []
        ltime = []
        
        for i in range(len(res)):
            cur_h = res['H{}'.format(MAX_DAYS)].iloc[i]
            cur_l = res['L{}'.format(MAX_DAYS)].iloc[i]
            if(cur_h != prev_h):
                prev_h = cur_h
                prev_h_time = res.Date_Time.iloc[i]
            if(cur_l != prev_l):
                prev_l = cur_l
                prev_l_time = res.Date_Time.iloc[i]
            htime.append(prev_h_time)
            ltime.append(prev_l_time)
        res['H{}_TIME'.format(MAX_DAYS)] = htime
        res['L{}_TIME'.format(MAX_DAYS)] = ltime
    dflist1.append(res)
    
df = pd.concat(dflist1,ignore_index=True)
df1 = df
df

In [ ]:
MAX_DAYS = 10

df = df2
gp = df.groupby('Date')
start_date = None

count = 0

dflist1 = []

for k,res in gp:
    count += 1
    print(k)
    if(count <= MAX_DAYS):
        res['H{}_TIME'.format(MAX_DAYS)] = [None for _ in range(len(res))]
        res['L{}_TIME'.format(MAX_DAYS)] = [None for _ in range(len(res))] 
    else:
        prev_h = res['H{}'.format(MAX_DAYS)].iloc[0]
        prev_l = res['L{}'.format(MAX_DAYS)].iloc[0]
        
        prev_h_time = None
        prev_l_time = None
        
        hdf = df[(df.Date_Time <= res.Date_Time.iloc[0])]
        hdf = hdf[hdf.High == prev_h]
        
        ldf = df[(df.Date_Time <= res.Date_Time.iloc[0])]
        ldf = ldf[ldf.Low == prev_l]
        
        # 2013-06-15 10:06:00
        
        # print(k)
        # print(prev_h)
        # print(hdf)
        
        # print(prev_h)
        
        if(len(hdf) > 1):
            prev_h_time = hdf.Date_Time.iloc[-1]
        else:
            prev_h_time = hdf.Date_Time.iloc[0]
        
        if(len(ldf) > 1):
            prev_l_time = ldf.Date_Time.iloc[-1]
        else:
            prev_l_time = ldf.Date_Time.iloc[0]
        
        htime = []
        ltime = []
        
        for i in range(len(res)):
            cur_h = res['H{}'.format(MAX_DAYS)].iloc[i]
            cur_l = res['L{}'.format(MAX_DAYS)].iloc[i]
            if(cur_h != prev_h):
                prev_h = cur_h
                prev_h_time = res.Date_Time.iloc[i]
            if(cur_l != prev_l):
                prev_l = cur_l
                prev_l_time = res.Date_Time.iloc[i]
            htime.append(prev_h_time)
            ltime.append(prev_l_time)
        res['H{}_TIME'.format(MAX_DAYS)] = htime
        res['L{}_TIME'.format(MAX_DAYS)] = ltime
    dflist1.append(res)
    
df = pd.concat(dflist1,ignore_index=True)
df2 = df
df

In [ ]:
# 1min return
df = df1
returns = []
returns.append(0)
for i in range(1,len(df)):
    cur_close = df.Close.iloc[i]
    prev_close = df.Close.iloc[i-1]
    ret = 100*((cur_close-prev_close)/prev_close)
    returns.append(ret)

df['Returns_1'] = returns

df1 = df



# 1min return
df = df2
returns = []
returns.append(0)
for i in range(1,len(df)):
    cur_close = df.Close.iloc[i]
    prev_close = df.Close.iloc[i-1]
    ret = 100*((cur_close-prev_close)/prev_close)
    returns.append(ret)

df['Returns_1'] = returns

df2 = df

In [ ]:
from statistics import stdev


SD_DAYS = 3

df = df2

gp = df.groupby('Date')

sd = []

dflist1 = []

for k,res in gp:
    print(k)
    if(len(sd) < SD_DAYS):
        sd.append(res.Returns_1.to_list())
        res['SD_{}'.format(SD_DAYS)] = [0 for _ in range(len(res))]
    else:
        ret = []
        for i in range(len(sd)):
            cret = sd[i]
            for j in cret:
                ret.append(j)
        print(stdev(ret)*100)
        print(len(sd))
        res['SD_{}'.format(SD_DAYS)] = [100*stdev(ret) for _ in range(len(res))]
        sd.pop(0)
        sd.append(res.Returns_1.to_list())
    dflist1.append(res)

df = pd.concat(dflist1,ignore_index=True)

df2 = df

from statistics import stdev


SD_DAYS = 3

df = df1

gp = df.groupby('Date')

sd = []

dflist1 = []

for k,res in gp:
    print(k)
    if(len(sd) < SD_DAYS):
        sd.append(res.Returns_1.to_list())
        res['SD_{}'.format(SD_DAYS)] = [0 for _ in range(len(res))]
    else:
        ret = []
        for i in range(len(sd)):
            cret = sd[i]
            for j in cret:
                ret.append(j)
        print(stdev(ret)*100)
        print(len(sd))
        res['SD_{}'.format(SD_DAYS)] = [100*stdev(ret) for _ in range(len(res))]
        sd.pop(0)
        sd.append(res.Returns_1.to_list())
    dflist1.append(res)

df = pd.concat(dflist1,ignore_index=True)

df1 = df

from math import sqrt


df2.SD_3 = (df2.SD_3)/100
df2.SD_3 = df2.SD_3 * sqrt(870) * 4

# df1['SD_3_15'] = df1['SD_3']
df2['SD_3_15'] = df2['SD_3']
# df1['SD_3_20'] = df1['SD_3']
df2['SD_3_20'] = df2['SD_3']

df = df2

for i in range(len(df)):
    if(df['SD_3'].iloc[i] > 15):
        df['SD_3_15'].iloc[i] = 15
    # if(df['SD_3'].iloc[i] > 20):
    #     df['SD_3_20'].iloc[i] = 20
    
df2 = df


df1.SD_3 = (df1.SD_3)/100
df1.SD_3 = df1.SD_3 * sqrt(870) * 4

# df1['SD_3_15'] = df1['SD_3']
df1['SD_3_15'] = df1['SD_3']
# df1['SD_3_20'] = df1['SD_3']
df1['SD_3_20'] = df1['SD_3']

df = df1

for i in range(len(df)):
    if(df['SD_3'].iloc[i] > 15):
        df['SD_3_15'].iloc[i] = 15
    # if(df['SD_3'].iloc[i] > 20):
    #     df['SD_3_20'].iloc[i] = 20
 
df1 = df





df = df2

MAX_DAYS = 10
SD_DAYS = 3
SD_LIMIT = 15
df1['Range_{}'.format(MAX_DAYS)] = 100*((df1['H{}'.format(MAX_DAYS)]-df1['L{}'.format(MAX_DAYS)])/(df1['H{}'.format(MAX_DAYS)]))
df2['Range_{}'.format(MAX_DAYS)] = 100*((df2['H{}'.format(MAX_DAYS)]-df2['L{}'.format(MAX_DAYS)])/(df2['H{}'.format(MAX_DAYS)]))

df['Theoritical H{}'.format(MAX_DAYS)] = [0]*len(df)
df['Theoritical L{}'.format(MAX_DAYS)] = [0]*len(df)

gp = df.groupby('Date')

dflist1 = []

count = 0

for k,res in gp:
    print(k)
    count += 1
    
    if(count <= MAX_DAYS):
        res['Theoritical H{}'.format(MAX_DAYS)] = res['H{}'.format(MAX_DAYS)]
        res['Theoritical L{}'.format(MAX_DAYS)] = res['L{}'.format(MAX_DAYS)]
        dflist1.append(res)
        continue
    for i in range(len(res)):
        cur_range = res['Range_{}'.format(MAX_DAYS)].iloc[i]
        cur_sd = res['SD_{}_{}'.format(SD_DAYS,SD_LIMIT)].iloc[i]
        if(cur_range > cur_sd): 
            if(res['Theoritical L{}'.format(MAX_DAYS)].iloc[i] != 0):
                if((res.Low.iloc[i] < res['Theoritical L{}'.format(MAX_DAYS)].iloc[i])):
                    res['Theoritical L{}'.format(MAX_DAYS)].iloc[i:] = res.Low.iloc[i]
                    res['Theoritical H{}'.format(MAX_DAYS)].iloc[i:] = (res['Theoritical L{}'.format(MAX_DAYS)].iloc[i])*(1+(cur_sd/100))
                elif((res.High.iloc[i] > res['Theoritical H{}'.format(MAX_DAYS)].iloc[i])):
                    res['Theoritical H{}'.format(MAX_DAYS)].iloc[i:] = res.High.iloc[i]
                    res['Theoritical L{}'.format(MAX_DAYS)].iloc[i:] = res['Theoritical H{}'.format(MAX_DAYS)].iloc[i]*(1-(cur_sd/100))
            else:
                if(res['H{}_TIME'.format(MAX_DAYS)].iloc[i] > res['L{}_TIME'.format(MAX_DAYS)].iloc[i]):
                    # Theoritical Low Change
                    tl = (res['H{}'.format(MAX_DAYS)].iloc[i])*(1-(cur_sd/100))
                    res['Theoritical L{}'.format(MAX_DAYS)].iloc[i:] = tl 
                    res['Theoritical H{}'.format(MAX_DAYS)].iloc[i:] = res['H{}'.format(MAX_DAYS)].iloc[i]
                else:
                    th = (res['L{}'.format(MAX_DAYS)].iloc[i])*(1+(cur_sd/100))
                    res['Theoritical H{}'.format(MAX_DAYS)].iloc[i:] = th 
                    res['Theoritical L{}'.format(MAX_DAYS)].iloc[i:] = res['L{}'.format(MAX_DAYS)].iloc[i]
        else:
            res['Theoritical H{}'.format(MAX_DAYS)].iloc[i] = res['H{}'.format(MAX_DAYS)].iloc[i]
            res['Theoritical L{}'.format(MAX_DAYS)].iloc[i] = res['L{}'.format(MAX_DAYS)].iloc[i]
    dflist1.append(res)
    
df = pd.concat(dflist1,ignore_index=True)

df2 = df



df=df1

MAX_DAYS = 10
SD_DAYS = 3
SD_LIMIT = 15
df1['Range_{}'.format(MAX_DAYS)] = 100*((df1['H{}'.format(MAX_DAYS)]-df1['L{}'.format(MAX_DAYS)])/(df1['H{}'.format(MAX_DAYS)]))
df2['Range_{}'.format(MAX_DAYS)] = 100*((df2['H{}'.format(MAX_DAYS)]-df2['L{}'.format(MAX_DAYS)])/(df2['H{}'.format(MAX_DAYS)]))

df['Theoritical H{}'.format(MAX_DAYS)] = [0]*len(df)
df['Theoritical L{}'.format(MAX_DAYS)] = [0]*len(df)

gp = df.groupby('Date')

dflist1 = []

count = 0

for k,res in gp:
    print(k)
    count += 1
    
    if(count <= MAX_DAYS):
        res['Theoritical H{}'.format(MAX_DAYS)] = res['H{}'.format(MAX_DAYS)]
        res['Theoritical L{}'.format(MAX_DAYS)] = res['L{}'.format(MAX_DAYS)]
        dflist1.append(res)
        continue
    for i in range(len(res)):
        cur_range = res['Range_{}'.format(MAX_DAYS)].iloc[i]
        cur_sd = res['SD_{}_{}'.format(SD_DAYS,SD_LIMIT)].iloc[i]
        if(cur_range > cur_sd): 
            if(res['Theoritical L{}'.format(MAX_DAYS)].iloc[i] != 0):
                if((res.Low.iloc[i] < res['Theoritical L{}'.format(MAX_DAYS)].iloc[i])):
                    res['Theoritical L{}'.format(MAX_DAYS)].iloc[i:] = res.Low.iloc[i]
                    res['Theoritical H{}'.format(MAX_DAYS)].iloc[i:] = (res['Theoritical L{}'.format(MAX_DAYS)].iloc[i])*(1+(cur_sd/100))
                elif((res.High.iloc[i] > res['Theoritical H{}'.format(MAX_DAYS)].iloc[i])):
                    res['Theoritical H{}'.format(MAX_DAYS)].iloc[i:] = res.High.iloc[i]
                    res['Theoritical L{}'.format(MAX_DAYS)].iloc[i:] = res['Theoritical H{}'.format(MAX_DAYS)].iloc[i]*(1-(cur_sd/100))
            else:
                if(res['H{}_TIME'.format(MAX_DAYS)].iloc[i] > res['L{}_TIME'.format(MAX_DAYS)].iloc[i]):
                    # Theoritical Low Change
                    tl = (res['H{}'.format(MAX_DAYS)].iloc[i])*(1-(cur_sd/100))
                    res['Theoritical L{}'.format(MAX_DAYS)].iloc[i:] = tl 
                    res['Theoritical H{}'.format(MAX_DAYS)].iloc[i:] = res['H{}'.format(MAX_DAYS)].iloc[i]
                else:
                    th = (res['L{}'.format(MAX_DAYS)].iloc[i])*(1+(cur_sd/100))
                    res['Theoritical H{}'.format(MAX_DAYS)].iloc[i:] = th 
                    res['Theoritical L{}'.format(MAX_DAYS)].iloc[i:] = res['L{}'.format(MAX_DAYS)].iloc[i]
        else:
            res['Theoritical H{}'.format(MAX_DAYS)].iloc[i] = res['H{}'.format(MAX_DAYS)].iloc[i]
            res['Theoritical L{}'.format(MAX_DAYS)].iloc[i] = res['L{}'.format(MAX_DAYS)].iloc[i]
    dflist1.append(res)
    
df = pd.concat(dflist1,ignore_index=True)

df1 = df


df1['K{}'.format(MAX_DAYS)] = 100*(df1.Close - df1['Theoritical L{}'.format(MAX_DAYS)])/(df1['Theoritical H{}'.format(MAX_DAYS)] - df1['Theoritical L{}'.format(MAX_DAYS)])
df2['K{}'.format(MAX_DAYS)] = 100*(df2.Close - df2['Theoritical L{}'.format(MAX_DAYS)])/(df2['Theoritical H{}'.format(MAX_DAYS)] - df2['Theoritical L{}'.format(MAX_DAYS)])



data = df1
gp = data.groupby('Date')
dflist = []
for k,res in gp:
    start_time = res.Date_Time.iloc[0]
    res = res[res.Date_Time >= start_time + timedelta(minutes = NO_TRADING)]
    dflist.append(res)
    
data = pd.concat(dflist,ignore_index=True)

df1 = data

data = df2
gp = data.groupby('Date')
dflist = []
for k,res in gp:
    start_time = res.Date_Time.iloc[0]
    res = res[res.Date_Time >= start_time + timedelta(minutes = NO_TRADING)]
    dflist.append(res)
    
data = pd.concat(dflist,ignore_index=True)

df2 = data







print(df1)
print(df2)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (20,15)

plt.plot(df1.Date_Time,df1.SD_3_20,color='b',label='Theoritical Range')
plt.plot(df1.Date_Time,df1.Range_10,color='g',label='Actual Range')

plt.plot(df1.Date_Time,[20]*len(df1.Date_Time),color='r')

plt.yticks(list(plt.yticks()[0]) + [20])

plt.legend()

In [ ]:
INDEX = [0,0] # USED AS A POINTER IN EACH DATA FRAME. TELLING US WHICH ROW TO LOOK IN. INDEX[0] = 1 MEANS WE ARE IN DF2 AT ROW (INDEX[1])
INTRADE = 0 # INTRADE = 1 (WE ARE CURRENTLY IN AN ONGOING TRADE)
ALL_DATA = 0 # ALL DATA HAS BEEN CHECKED AND WE HAVE REACHED THE END OF THE DATA

POSITION_LONG = [0]*len(STOCH_BUY) # POSITION_LONG = [0,1] MEANS WE HAVE AN OPEN POSITION CORRESPONDING TO STOCH VALUE 95. [1,1] MEANS WE HAVE OPEN POISTIONS WORTH 1 CR FOR STOCH VALUE 80 AND 95
POSITION_SHORT = [0]*len(STOCH_SELL) # SAME AS ABOVE

BUY = [] # EVERY TIME WE GO LONG, WE WILL ADD THE BUY PRICE HERE. THIS WILL LATER BE USED TO CALCULATE PNL
SELL = [] # SAME AS BUY 

BUY_SL = [None]*len(STOCH_BUY) # STORES THE STOP LOSS (IN RS.) FOR EVERY LONG POSITION OPENED
SELL_SL = [None]*len(STOCH_SELL) # SAME AS BUY_SL

BUY_LOTS = [0]*len(STOCH_BUY) # STORES THE LOTS OPENED FOR EVERY LONG POSITION OPENED
SELL_LOTS = [0]*len(STOCH_SELL)

MAX_RISK_TAKEN = 0

PREV_SL_UPD_TIME = None # VARIABLE TO STORE WHEN THE LAST TIME STOP LOSS WAS UPDATED. 

tradeID = 0 # FOR EVERY TRADE INITIATED, IT WILL HAVE A UNIQUE TRADE ID
trades = pd.DataFrame()

SL_DT = []
SL = []



In [ ]:
# FUNCTIONS UPDATES THE STOP LOSS FOR OPEN POSITIONS

def update_SL(data,index):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE,SL_DT,SL
    cur_high = data['Theoritical H{}'.format(MAX_DAYS)].iloc[index]
    cur_low = data['Theoritical L{}'.format(MAX_DAYS)].iloc[index]
    cur_price = data.Close.iloc[index]
    for i in range(len(POSITION_LONG)):
        if(POSITION_LONG[i]):
            perc = STOCH_BUY[i] - SL_PERCENTILE
            sl = cur_low + (perc/100)*(cur_high-cur_low)
            if((abs(cur_price - sl) / cur_price)*100 > MAX_SL):
                sl = cur_price - (MAX_SL/100)*cur_price
            elif((abs(cur_price - sl) / cur_price)*100 < MIN_SL):
                sl = cur_price - (MIN_SL/100)*cur_price
            # BUY_SL[i] = max(BUY_SL[i],sl)
            # BUY_SL[i] = sl
            if(BUY_SL[i] != None):
                if(sl > BUY_SL[i]):
                    SL_DT.append(data.Date_Time.iloc[index])
                    p = 100*(abs(sl - cur_price))/(min(sl,cur_price))
                    SL.append(p)
                BUY_SL[i] = max(BUY_SL[i],sl)
                
            else:
                SL_DT.append(data.Date_Time.iloc[index])
                p = 100*(abs(sl - cur_price))/(min(sl,cur_price))
                SL.append(p)
                BUY_SL[i] = sl
        if(POSITION_SHORT[i]):
            perc = STOCH_SELL[i] + SL_PERCENTILE
            sl = cur_low + (perc/100)*(cur_high - cur_low)
            if((abs(cur_price - sl) / cur_price)*100 > MAX_SL):
                sl = cur_price + (MAX_SL/100)*cur_price
            elif((abs(cur_price - sl) / cur_price)*100 < MIN_SL):
                sl = cur_price + (MIN_SL/100)*cur_price  
            # SELL_SL[i] = sl
            if(SELL_SL[i] != None and SELL_SL[i] != 0):
                if(sl < SELL_SL[i]):
                    SL_DT.append(data.Date_Time.iloc[index])
                    p = 100*(abs(sl - cur_price))/(min(sl,cur_price))
                    SL.append(p)
                SELL_SL[i] = min(SELL_SL[i],sl)
            else:
                SL_DT.append(data.Date_Time.iloc[index])
                p = 100*(abs(sl - cur_price))/(min(sl,cur_price))
                SL.append(p)
                SELL_SL[i] = sl  
    if(sum(POSITION_LONG)):
            print("SL Updated at time " + str(data.Date_Time.iloc[index]))
            PREV_SL_UPD_TIME = pd.to_datetime(data.Date_Time.iloc[index])
            print(BUY_SL)
    if(sum(POSITION_SHORT)):
            print("SL Updated at time " + str(data.Date_Time.iloc[index]))
            PREV_SL_UPD_TIME = pd.to_datetime(data.Date_Time.iloc[index])
            print(SELL_SL)

# AFTER CLOSING EVERY TRADE, WE INITIALIZE THE VARIABLES

def initialize():
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE,MAX_RISK_TAKEN
    
    INTRADE = 0

    POSITION_LONG = [0]*len(STOCH_BUY)
    POSITION_SHORT = [0]*len(STOCH_SELL)

    BUY = [] 
    SELL = []

    BUY_SL = [None]*len(STOCH_BUY)
    SELL_SL = [None]*len(STOCH_SELL)

    BUY_LOTS = [0]*len(STOCH_BUY)
    SELL_LOTS = [0]*len(STOCH_SELL)
    
    MAX_RISK_TAKEN = 0
    
# JUMP CONDITIONS FOR ROLLOVER

def jump_conditions(data,index):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE,ROLLOVER_TIME
    dte = data.DTE.iloc[index]
    cur_time = pd.to_datetime(data.Date_Time.iloc[index])
    cur_time = cur_time.time()
    
    if(INDEX[0] and dte > ROLLOVER_DAYS):
        return True
    if(INDEX[0] == 0 and dte <= ROLLOVER_DAYS and cur_time >= ROLLOVER_TIME):
        return True
    
    return False    

# WHENEVER WE ARE JUMPING FROM DF1 TO DF2, WE CALL ROLLOVER TO CLOSE OPEN POSITIONS IN DF1 AND OPEN IN DF2. 

def rollover(data1,data2,i):
    global SELL_SL,BUY_SL,POSITION_LONG,POSITION_SHORT
    if(not INDEX[0]):
        None     
    else:
        h1 = data1['Theoritical H{}'.format(MAX_DAYS)].iloc[i]
        l1 = data1['Theoritical L{}'.format(MAX_DAYS)].iloc[i]
        h2 = data1['Theoritical H{}'.format(MAX_DAYS)].iloc[0]
        l2 = data1['Theoritical L{}'.format(MAX_DAYS)].iloc[0]
        
        if(sum(POSITION_LONG)):
            print("<<< ROLLING OVER >>>")
            lots = sum(BUY_LOTS)
            closing_price = data1.Close.iloc[i]
            opening_price = data2.Close.iloc[0]
            print("Sold " + str(lots) + " at a price of " + str(closing_price))
            print("Current Time : " + str(data1.Date_Time.iloc[i]))
            journal_trade(data1.Ticker.iloc[i],tradeID,"Rollover",data1.Date_Time.iloc[i],"Short",0,closing_price,lots,0,BALANCE,h1,l1,0)
            
            for i in range(lots):
                SELL.append(closing_price)
                BUY.append(opening_price)
                
            print("Bought " + str(lots) + " at a price of " + str(opening_price))
            print("Current Time : " + str(data2.Date_Time.iloc[0]))
            print("<<< ROLLING OVER COMPLETE >>>")
            for i in range(len(BUY_SL)):
                if(POSITION_LONG[i]):
                    BUY_SL[i] = 0
            update_SL(data2,0)
            journal_trade(data2.Ticker.iloc[i],tradeID,"Rollover",data2.Date_Time.iloc[0],"Long",0,opening_price,lots,0,BALANCE,h2,l2,0)
            
        elif(sum(POSITION_SHORT)):
            print("<<< ROLLING OVER >>>")
            lots = sum(SELL_LOTS)
            closing_price = data1.Close.iloc[i]
            opening_price = data2.Close.iloc[0]
            print("Bought " + str(lots) + " at a price of " + str(closing_price))
            print("Current Time : " + str(data1.Date_Time.iloc[i]))
            journal_trade(data1.Ticker.iloc[i],tradeID,"Rollover",data1.Date_Time.iloc[i],"Long",0,closing_price,lots,0,BALANCE,h1,l1,0)
            for i in range(lots):
                BUY.append(closing_price)
                SELL.append(opening_price)
                
            print("Sold " + str(lots) + " at a price of " + str(opening_price))
            print("Current Time : " + str(data2.Date_Time.iloc[0]))
            print("<<< ROLLING OVER COMPLETE >>>")
            for i in range(len(SELL_SL)):
                if(POSITION_SHORT[i]):
                    SELL_SL[i] = 100000
            update_SL(data2,0)
            journal_trade(data2.Ticker.iloc[i],tradeID,"Rollover",data2.Date_Time.iloc[0],"Short",0,opening_price,lots,0,BALANCE,h2,l2,0)
        return

# A FUNCTION TO JOURNAL TRADES IN A CSV

def journal_trade(symbol,id,strategy,date_time,type,stoch,price,lots,result,balance,h,l,rr):
    global trades
    
    tmp = pd.DataFrame({
        "Symbol" : symbol,
        "TradeID" : id,
        "Strategy" : strategy,
        "Date_Time" : date_time,
        "Type" : type,
        "Stochastic Indicator" : stoch,
        "Price" : price,
        "Lots" : lots,
        'High' : h,
        'Low':l,
        "Result" : result,
        "Balance" : balance,
        "Risk_Reward" : rr,
    },index ={0})
    frames = [trades,tmp]
    trades = pd.concat(frames)

In [ ]:
from statistics import mean

# WILL RETURN TRUE WHEN WE SATISFY LONG STRATEGY CONDITIONS
def buy_conditions_long(data,index):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE
    if(index >= len(data.Open) - 1):
        return False
    
    first = 0
    
    cur_stoch = data['K' + str(MAX_DAYS)].iloc[index]
    
    for i in range(len(POSITION_LONG)):
        if(POSITION_LONG[i]):
            first = i
    i = first 
    
    while(i < len(STOCH_BUY)):
        if(POSITION_LONG[i] == 0 and cur_stoch >= STOCH_BUY[i]):
            return True
        i = i + 1    
    return False

# WILL RETURN TRUE WHEN WE HIT STOP LOSS IN A LONG STRATEGY
def sell_condition_long(data,index):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE
    if(index >= len(data.Open) - 1):
        return True
    cur_stoch = data['K' + str(MAX_DAYS)].iloc[index]
    cur_low = data.Low.iloc[index]
    for i in range(len(STOCH_BUY)):
        if(POSITION_LONG[i] == 1 and cur_low <= BUY_SL[i]):
            return True
    return False

# IF WE SATISFY BUY CONDITIONS FOR A LONG STRATEGY, THIS FUNCTION WILL BUY LOTS AT A GIVEN PRICE. 
def buy_signal_long(data,index):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE,tradeID,RISK_PER_POSITION,MAX_RISK_TAKEN
    
    first = 0
    
    cur_stoch = data['K' + str(MAX_DAYS)].iloc[index]
    
    for i in range(len(POSITION_LONG)):
        if(POSITION_LONG[i]):
            first = i
    i = first 
    
    target = first
    
    while(i < len(STOCH_BUY)):
        if(POSITION_LONG[i] == 0 and cur_stoch >= STOCH_BUY[i]):
            target = i
        i = i + 1     
        
    cur_price = data.Close.iloc[index]
    
    POSITION_LONG[target] = 1
    update_SL(data,index)
    
    # cur_lots = max(int(MAX_ALLOCATION/(cur_price*LOT_SIZE)),1)    
    # cur_lots = min(cur_lots,MAX_LOTS)
    
    risk = cur_price - BUY_SL[target]
    
    MAX_RISK_TAKEN = max(MAX_RISK_TAKEN,len(POSITION_LONG) * RISK_PER_POSITION)
    
    cur_lots = int(RISK_PER_POSITION / (LOT_SIZE * risk))
    
    for i in range(cur_lots):
        BUY.append(cur_price)

    
    BUY_LOTS[target] = cur_lots
    
    if(INTRADE == 0):
        INTRADE = 1
        tradeID = tradeID + 1
        print("<<< NEW LONG TRADE STARTED >>>")
    
    print("Bought " + str(cur_lots) + " at a price of " + str(cur_price))
    print("Current Stoch Value : " + str(cur_stoch))
    print("Current Time : " + str(data.Date_Time.iloc[index]))
    print(BUY_LOTS)
    print(POSITION_LONG)
    
    h = data['Theoritical H{}'.format(MAX_DAYS)].iloc[index]
    l = data['Theoritical L{}'.format(MAX_DAYS)].iloc[index]
    
    

    journal_trade(data.Ticker.iloc[index],tradeID,"Long Strategy",data.Date_Time.iloc[index],"Long",cur_stoch,cur_price,cur_lots,0,BALANCE,h,l,0)
    
    return


# ON HITTING SL, WE SELL OUR LONG POSITIONS.
def sell_signal_long(data,index):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE,tradeID,MAX_RISK_TAKEN
    
    cur_stoch = data['K' + str(MAX_DAYS)].iloc[index]
    cur_low = data.Low.iloc[index]
    
    target = 0
    
    for i in range(len(STOCH_BUY)):
        if(POSITION_LONG[i] == 1 and cur_low <= BUY_SL[i]):
            target = i
            break
            
    i = target 
    
    cur_lots = sum(BUY_LOTS[i:])
    cur_price = data.Close.iloc[index]
    cur_time = data.Date_Time.iloc[index]
    
    for j in range(cur_lots):
        SELL.append(cur_price)
    
    while(i < len(POSITION_LONG)):
        POSITION_LONG[i] = 0
        BUY_LOTS[i] = 0
        BUY_SL[i] = 0
        i = i + 1
        
    print("Sold " + str(cur_lots) + " at a price of " + str(cur_price))
    print("Current Stoch Value : " + str(cur_stoch))
    print("Current Time : " + str(data.Date_Time.iloc[index]))
    print(BUY_LOTS)
    print(POSITION_LONG)
    
    res = 0 
    
    rr = 0
    
    if(sum(BUY_LOTS) == 0):
        if(len(BUY) != len(SELL)):
            print("ERROR IN LENGTH!")
            
        # COST CALCULATION : 
        cost = 0
        for i in BUY:
            posi = i*LOT_SIZE
            cost = cost + (posi * COST_PERCENT)/100
        for i in SELL:
            posi = i*LOT_SIZE
            cost = cost + (posi * COST_PERCENT)/100
        
        
        PNL = len(BUY)*(mean(SELL) - mean(BUY))*LOT_SIZE - cost
        BALANCE = BALANCE + PNL
        
        rr = round(PNL / MAX_RISK_TAKEN,2)
        
        res = PNL
        print("<<< LONG TRADE CLOSED >>>")
        
        print("")
        print("PNL : " + str(PNL))
        print("COST : " + str(cost))
        print("BALANCE : " + str(BALANCE))
        print("")
        
        
        
        initialize()
    
    h = data['Theoritical H{}'.format(MAX_DAYS)].iloc[index]
    l = data['Theoritical L{}'.format(MAX_DAYS)].iloc[index]
    
    journal_trade(data.Ticker.iloc[index],tradeID,"Long Strategy",data.Date_Time.iloc[index],"Short",cur_stoch,cur_price,cur_lots,res,BALANCE,h,l,rr)
    
    update_SL(data,index)
        
# THIS FUNCTION WILL RUN UNTIL A LONG STRATEGY TRADE IS SQUARED OFF. IT WILL INITIATE BUYS AND SELLS.

def long_strategy(data1,data2):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE,df1,df2
    
    if(ALL_DATA):
        return
    
    i = INDEX[1]
    data = data1
    if(INDEX[0]):
        data = data2
    
    if(jump_conditions(data,i)):
        if(INDEX[0]):
            print("DATA2 ---> DATA1 AT " + str(data2.Date_Time.iloc[i]))
            # data1 = data1.where(data1.Date_Time >= data2.Date_Time.iloc[i])
            # data1 = data1.dropna(axis=0)
            data1 = data1[data1.Date_Time >= data2.Date_Time.iloc[i]]
            INDEX = [0,0]               
            df1 = data1
        else:
            print("DATA1 ---> DATA2 AT " + str(data1.Date_Time.iloc[i]))
            # data2 = data2.where(data2.Date_Time >= data1.Date_Time.iloc[i])
            # data2 = data2.dropna(axis=0)
            data2 = data2[data2.Date_Time >= data1.Date_Time.iloc[i]]
            INDEX = [1,0]
            print(data2)
            df2 = data2
            rollover(data1,data2,i)     
        
        return long_strategy(data1,data2)
    
    length = len(data.Close)
    
    while(i < length):
        # INDEX[1] = i
        # print("INDEX UPDATE TO : " + str(INDEX))
        if(jump_conditions(data,i)):
            if(INDEX[0]):
                print("DATA2 ---> DATA1 AT " + str(data2.Date_Time.iloc[i]))
                # data1 = data1.where(data1.Date_Time >= data2.Date_Time.iloc[i])
                # data1 = data1.dropna(axis=0)
                data1 = data1[data1.Date_Time >= data2.Date_Time.iloc[i]]
                INDEX = [0,0]               
                df1 = data1
            else:
                print("DATA1 ---> DATA2 AT " + str(data1.Date_Time.iloc[i]))
                # data2 = data2.where(data2.Date_Time >= data1.Date_Time.iloc[i])
                # data2 = data2.dropna(axis=0)
                data2 = data2[data2.Date_Time >= data1.Date_Time.iloc[i]]
                INDEX = [1,0]
                print(data2)
                df2 = data2
                rollover(data1,data2,i)     
            return long_strategy(data1,data2)
        
        cur_time = data.Date_Time.iloc[i]
        if(PREV_SL_UPD_TIME != None and (pd.to_datetime(cur_time) - pd.to_datetime(PREV_SL_UPD_TIME)).total_seconds() >= SL_CHANGE):
            update_SL(data,i)
        
        if(buy_conditions_long(data,i)):
            buy_signal_long(data,i)
        elif(sell_condition_long(data,i) and INTRADE == 1):
            sell_signal_long(data,i)
            if(INTRADE == 0):
                print("INDEX UPDATED TO " + str(data.Date_Time.iloc[i]))
                INDEX[1] = i
                # if(INDEX[0]):
                #     print("DF START : ")
                #     print(data2.Date_Time.iloc[0])
                # else:
                #     print("DF START : ")
                #     print(data1.Date_Time.iloc[0])
                return                
        i = i + 1
    ALL_DATA = 1
    
    

In [ ]:
from statistics import mean

# SELL CONDITIONS FOR A SHORT STRATEGY 
def sell_condition_short(data,index):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE
    if(index >= len(data.Open) - 1):
        return False
    
    first = 0
    
    cur_stoch = data['K' + str(MAX_DAYS)].iloc[index]
    
    for i in range(len(POSITION_SHORT)):
        if(POSITION_SHORT[i]):
            first = i
    i = first 
    
    while(i < len(STOCH_SELL)):
        if(POSITION_SHORT[i] == 0 and cur_stoch <= STOCH_SELL[i]):
            return True
        i = i + 1    
    return False

# SL CONDITIONS FOR A SHORT STRATEGY

def buy_condition_short(data,index):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE
    if(index >= len(data.Open) - 1):
        return True
    cur_stoch = data['K' + str(MAX_DAYS)].iloc[index]
    cur_high = data.High.iloc[index]
    # for i in range(len(STOCH_BUY)):
    #     if(POSITION_LONG[i] == 1 and cur_stoch < (STOCH_BUY[i] - SL_PERCENTILE)):
            # return True
    for i in range(len(STOCH_SELL)):
        if(POSITION_SHORT[i] == 1 and cur_high >= SELL_SL[i]):
            return True
    return False


# INITIATE SELLING FOR A SHORT STRATEGY
def sell_signal_short(data,index):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE,tradeID,RISK_PER_POSTION,MAX_RISK_TAKEN
    
    first = 0
    
    cur_stoch = data['K' + str(MAX_DAYS)].iloc[index]
    
    for i in range(len(POSITION_SHORT)):
        if(POSITION_SHORT[i]):
            first = i
    i = first 
    
    target = first
    
    while(i < len(STOCH_SELL)):
        if(POSITION_SHORT[i] == 0 and cur_stoch <= STOCH_SELL[i]):
            target = i
        i = i + 1     
        
    cur_price = data.Close.iloc[index]
    POSITION_SHORT[target] = 1
    update_SL(data,index)
    
    risk = SELL_SL[target] - cur_price
    
    cur_lots = int(RISK_PER_POSITION / (LOT_SIZE * risk))
    
    # cur_lots = max(int(MAX_ALLOCATION/(cur_price*LOT_SIZE)),1)    
    # cur_lots = min(cur_lots,MAX_LOTS)
    
    MAX_RISK_TAKEN = max(MAX_RISK_TAKEN,len(POSITION_SHORT)*RISK_PER_POSITION)
    
    for i in range(cur_lots):
        SELL.append(cur_price)

    
    SELL_LOTS[target] = cur_lots
    
    if(INTRADE == 0):
        INTRADE = 1
        tradeID += 1
        print("<<< NEW SHORT TRADE STARTED >>>")
    
    print("Sold " + str(cur_lots) + " at a price of " + str(cur_price))
    print("Current Stoch Value : " + str(cur_stoch))
    print("Current Time : " + str(data.Date_Time.iloc[index]))
    print(SELL_LOTS)
    print(POSITION_SHORT)
    
    h = data['Theoritical H{}'.format(MAX_DAYS)].iloc[index]
    l = data['Theoritical L{}'.format(MAX_DAYS)].iloc[index]
    
    
    journal_trade(data.Ticker.iloc[index],tradeID,"Short Strategy",data.Date_Time.iloc[index],"Short",cur_stoch,cur_price,cur_lots,0,BALANCE,h,l,0)
    return
    
# INITIATE SQUARING OFF FOR A SHORT STRATEGY
def buy_signal_short(data,index):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE,MAX_RISK_TAKEN
    
    cur_stoch = data['K' + str(MAX_DAYS)].iloc[index]
    cur_high = data.High.iloc[index]
    
    target = 0
    
    for i in range(len(STOCH_SELL)):
        if(POSITION_SHORT[i] == 1 and cur_high >= SELL_SL[i]):
            target = i
            break
            
    i = target 
    
    cur_lots = sum(SELL_LOTS[i:])
    cur_price = data.Close.iloc[index]
    cur_time = data.Date_Time.iloc[index]
    
    for j in range(cur_lots):
        BUY.append(cur_price)
    
    while(i < len(POSITION_SHORT)):
        POSITION_SHORT[i] = 0
        SELL_LOTS[i] = 0
        SELL_SL[i] = 0
        i = i + 1
        
    print("Bought " + str(cur_lots) + " at a price of " + str(cur_price))
    print("Current Stoch Value : " + str(cur_stoch))
    print("Current Time : " + str(data.Date_Time.iloc[index]))
    print(SELL_LOTS)
    print(POSITION_SHORT)
    
    res = 0
    
    rr = 0
    
    if(sum(SELL_LOTS) == 0):
        if(len(BUY) != len(SELL)):
            print("ERROR IN LENGTH!")
            
        # COST CALCULATION : 
        cost = 0
        for i in BUY:
            posi = i*LOT_SIZE
            cost = cost + (posi * COST_PERCENT)/100
        for i in SELL:
            posi = i*LOT_SIZE
            cost = cost + (posi * COST_PERCENT)/100    
            
        PNL = len(BUY)*(mean(SELL) - mean(BUY))*LOT_SIZE - cost
        BALANCE = BALANCE + PNL
        res = PNL
        print("<<< LONG TRADE CLOSED >>>")
        
        rr = round(PNL/MAX_RISK_TAKEN,2)
        
        print("")
        print("PNL : " + str(PNL))
        print("COST : " + str(cost))
        print("BALANCE : " + str(BALANCE))
        print("")

        initialize()
    
    h = data['Theoritical H{}'.format(MAX_DAYS)].iloc[index]
    l = data['Theoritical L{}'.format(MAX_DAYS)].iloc[index]
    
    update_SL(data,index)
    journal_trade(data.Ticker.iloc[index],tradeID,"Short Strategy",data.Date_Time.iloc[index],"Long",cur_stoch,cur_price,cur_lots,res,BALANCE,h,l,rr)


# SAME AS LONG_STRATEGY
def short_strategy(data1,data2):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE,df1,df2
    
    if(ALL_DATA):
        return
    
    i = INDEX[1]
    data = data1
    if(INDEX[0]):
        data = data2
    
    if(jump_conditions(data,i)):
        if(INDEX[0]):
            # data1 = data1.where(data1.Date_Time >= data2.Date_Time.iloc[i])
            # data1 = data1.dropna(axis=0)
            data1 = data1[data1.Date_Time >= data2.Date_Time.iloc[i]]
            INDEX = [0,0]     
            df1 = data1
            df2 = data2          
            print("DATA2 ---> DATA1 AT " + str(data2.Date_Time.iloc[i]))        
        else:
            # data2 = data2.where(data2.Date_Time >= data1.Date_Time.iloc[i])
            # data2 = data2.dropna(axis=0)
            data2 = data2[data2.Date_Time >= data1.Date_Time.iloc[i]]
            
            INDEX = [1,0]
            df1 = data1
            df2 = data2
            print("DATA1 ---> DATA2 AT " + str(data1.Date_Time.iloc[i]))
            rollover(data1,data2,i)     
        return short_strategy(data1,data2)
    
    length = len(data.Close)
    
    while(i < length):
        # INDEX[1] = i
        # print("INDEX UPDATE TO : " + str(INDEX))
        if(jump_conditions(data,i)):
            if(INDEX[0]):
                # data1 = data1.where(data1.Date_Time >= data2.Date_Time.iloc[i])
                # data1 = data1.dropna(axis=0)
                data1 = data1[data1.Date_Time >= data2.Date_Time.iloc[i]]
                INDEX = [0,0]     
                df1 = data1
                df2 = data2          
                print("DATA2 ---> DATA1 AT " + str(data2.Date_Time.iloc[i]))        
            else:
                # data2 = data2.where(data2.Date_Time >= data1.Date_Time.iloc[i])
                # data2 = data2.dropna(axis=0)
                data2 = data2[data2.Date_Time >= data1.Date_Time.iloc[i]]
                
                INDEX = [1,0]
                df1 = data1
                df2 = data2
                print("DATA1 ---> DATA2 AT " + str(data1.Date_Time.iloc[i]))
                rollover(data1,data2,i)     
            return short_strategy(data1,data2)
        
        cur_time = data.Date_Time.iloc[i]
        if(PREV_SL_UPD_TIME != None and (pd.to_datetime(cur_time) - pd.to_datetime(PREV_SL_UPD_TIME)).total_seconds() >= SL_CHANGE):
            update_SL(data,i)
        
        if(sell_condition_short(data,i)):
            sell_signal_short(data,i)
        elif(buy_condition_short(data,i) and INTRADE == 1):
            buy_signal_short(data,i)
            if(INTRADE == 0):
                INDEX[1] = i
                return                
        i = i + 1
    ALL_DATA = 1
    

In [ ]:
# WHEN NO TRADE IS GOING ON WE WILL ITERATE OVER ALL THE CANDLES UNTIL WE FIND A LONG STRAEGY OR A SHORT STRATEGY TRADE AND CALL LONG_STRATEGY OR SHORT_STRAEGY RESPECTIVELY.

def trade(data1,data2):
    global INDEX,INTRADE,ALL_DATA,POSITION_LONG,POSITION_SHORT,BUY,SELL,BUY_SL,SELL_SL,BUY_LOTS,SELL_LOTS,MAX_DAYS,SL_PERCENTILE,STOCH_BUY,ALLOCATION_BUY,STOCH_SELL,ALLOCATION_SELL,BALANCE,MAX_ALLOCATION,LEVERAGE,COST_PERCENT,ROLLOVER_DAYS,PREV_SL_UPD_TIME,SL_CHANGE,df1,df2
    
    if(ALL_DATA):
        return
    
    i = INDEX[1]
    data=data1
    if(INDEX[0]):
        data = data2
        
    if(i >= len(data.Close)-1):
        ALL_DATA = 1
        return
    
    
    
    if(jump_conditions(data,i)):
        if(INDEX[0]):
            # data1 = data1.where(data1.Date_Time >= data2.Date_Time.iloc[i])
            # data1 = data1.dropna(axis=0)
            data1 = data1[data1.Date_Time >= data2.Date_Time.iloc[i]]
            INDEX = [0,0]
            df1 = data1               
            print("DATA2 ---> DATA1 AT " + str(data2.Date_Time.iloc[i]))        
        else:
            # data2 = data2.where(data2.Date_Time >= data1.Date_Time.iloc[i])
            # data2 = data2.dropna(axis=0)
            data2 = data2[data2.Date_Time >= data1.Date_Time.iloc[i]]
            INDEX = [1,0]
            df2 = data2
            print("DATA1 ---> DATA2 AT " + str(data1.Date_Time.iloc[i]))
            rollover(data1,data2,i)     
        return trade(data1,data2)
    
    length = len(data.Close)
    
    while(i < len(data.Close)):
        # INDEX[1] = i
        # print("INDEX UPDATE TO : " + str(INDEX))
        if(jump_conditions(data,i)):
            if(INDEX[0]):
                # data1 = data1.where(data1.Date_Time >= data2.Date_Time.iloc[i])
                # data1 = data1.dropna(axis=0)
                data1 = data1[data1.Date_Time >= data2.Date_Time.iloc[i]]
                INDEX = [0,0]
                df1 = data1               
                print("DATA2 ---> DATA1 AT " + str(data2.Date_Time.iloc[i]))        
            else:
                # data2 = data2.where(data2.Date_Time >= data1.Date_Time.iloc[i])
                # data2 = data2.dropna(axis=0)
                data2 = data2[data2.Date_Time >= data1.Date_Time.iloc[i]]
                INDEX = [1,0]
                df2 = data2
                print("DATA1 ---> DATA2 AT " + str(data1.Date_Time.iloc[i]))
                rollover(data1,data2,i)     
            return trade(data1,data2)
        
        cur_time = data.Date_Time.iloc[i]
        
        if(buy_conditions_long(data,i)):
            long_strategy(data1,data2)
            i = INDEX[1]
            data1 = df1
            data2 = df2
            if(INDEX[0]):
                data = data2
                print("IN DATA 2 AFTER TRADE!")
                # if(i >= len(data.Close) - 1):
                #     return
                # print("INDEX : " + str(data.Date_Time.iloc[INDEX[1]]))
            else:
                data = data1
                print("IN DATA 1 AFTER TRADE!")
                # if(i >= len(data.Close) - 1):
                #     return
                # print("INDEX : " + str(data.Date_Time.iloc[INDEX[1]]))
            
        elif(sell_condition_short(data,i)):
            # INDEX[1] = i
            short_strategy(data1,data2)              
            i = INDEX[1]
            data1 = df1
            data2 = df2
            if(INDEX[0]):
                data = data2
                print("IN DATA 2 AFTER TRADE!")
            else:
                data = data1
                print("IN DATA 1 AFTER TRADE!")
        i = i + 1
    ALL_DATA = 1

In [ ]:
trade(df1,df2)

In [ ]:
trades = trades.reset_index()
trades.set_index('TradeID',inplace=True)
trades.drop(['index'],axis = 1 ,inplace=True)
trades.to_csv('./'+ VERSION+ '/' +COMMODITY_NAME+"/" + "Trades_v3_5min_{}_SD_{}_SL_{}_NOMAXLOTS.csv".format(MAX_DAYS,SD_LIMIT,35))
# trades.to_csv('./'+ VERSION+ '/' +COMMODITY_NAME+"/" + "Trades_v3_5min_{}_SD_{}.csv".format(MAX_DAYS,SD_LIMIT))
# # trades.to_csv('./CRUDEOIL_10DAYS_2015-2021.csv')

In [ ]:
# BASE_PATH = r'C:\Users\dhruv.oza\Desktop\C2_Fill\V2'

DAYS = 10
SD_LIMIT = 15
SL = '35'
VERSION = "V3_3_95"
COMMODITY_NAME = "CRUDEOIL"

LOT_SIZE = 100

BASE_PATH = './{}/{}'.format(VERSION,COMMODITY_NAME)
# BASE_PATH = './'
VERSION1 = "_v3_5min"

COMMODITY_NAME = 'CRUDEOIL'
DFX = './' + COMMODITY_NAME + './DATA_' + COMMODITY_NAME + '/' + COMMODITY_NAME + '-1.csv' 

MONTHLY_CSV = BASE_PATH + '/Monthly' + VERSION1 +'_{}_SD_{}_SL_{}_NOMAXLOTS.csv'.format(DAYS,SD_LIMIT,SL)
TRADES_CSV = BASE_PATH + '/Trades'+  VERSION1 +'_{}_SD_{}_SL_{}_NOMAXLOTS.csv'.format(DAYS,SD_LIMIT,SL)
# TRADES_CSV = r'C:\Users\dhruv.oza\Desktop\C2_Fill\CRUDEOIL_10DAYS_2015-2021.csv'
STATS_CSV = BASE_PATH + '/Stats' + VERSION1 +'_{}_SD_{}_SL_{}_NOMAXLOTS.csv'.format(DAYS,SD_LIMIT,SL)
YEAR_PLOTS = BASE_PATH + '/Year Plots' 
EQUITY_CURVE = BASE_PATH + '/Equity_Curve' + VERSION1 +'_{}_SD_{}_SL_{}_NOMAXLOTS.jpg'.format(DAYS,SD_LIMIT,SL)
EQUITY_CURVE_2 = BASE_PATH + '/EquityVSUnderlying' + VERSION1 +'_{}_SD_{}_SL_{}_NOMAXLOTS.jpg'.format(DAYS,SD_LIMIT,SL)

In [ ]:
from cmath import isnan
from numpy import NaN
import pandas as pd
import csv

rows = []


df = pd.read_csv(TRADES_CSV)

df_x = pd.read_csv(DFX)
# # df1 = pd.read_csv('Buy_Strategy_Trades.csv')
# # df2 = pd.read_csv('Sell_Strategy_Trades.csv')

# id = 1
# lst = []


# for i in range(len(df['Balance'])):
#     lst.append(id)
#     if(not isnan(df['Balance'].iloc[i]) and df.TradeType.iloc[i] != "Rollover"):
#         id = id + 1

# df['TradeId'] = lst  

# df['Date_Time']= pd.to_datetime(df['Date_Time'])      
# df.reset_index()
# df.set_index('TradeId',inplace=True)
# df.to_csv(TRADES_CSV)



print(df)

balance = []
strategy = []
pnl = []
dt = []
dt1 = []
dt2 = []
balance1 = []
balance2 = []

for i in range(len(df['Balance'])):
    if(isnan(df.Balance.iloc[i])):
        None
    else:
        strategy.append(df.Strategy.iloc[i])       
        balance.append(df.Balance.iloc[i])
        dt.append(df.Date_Time.iloc[i])
        pnl.append(df.Result.iloc[i])



        
data = pd.DataFrame()

data['Date_Time'] = dt
data['Strategy'] = strategy
data['Price'] = df['Price']
data['Result'] = pnl
data['Balance'] = balance

# data.to_csv('BalanceSheet.csv')

In [ ]:
from numpy import average
import numpy as np

exposure_df = df[::2]

Exposure = (exposure_df.Lots * exposure_df.Price * LOT_SIZE).to_list()

print("Average Exposure Taken : {}".format(average(Exposure)))
print("Minimum Exposure : {}".format(min(Exposure)))
print("Maximum Exposure : {}".format(max(Exposure)))
print("75th Percentile Exposure : {}".format(np.percentile(Exposure,75)))

rows.append(["Average Exposure Taken :" ,average(Exposure)])
rows.append(["Minimum Exposure :" ,min(Exposure)])
rows.append(["Maximum Exposure :" ,max(Exposure)])
rows.append(["75th Percentile Exposure :" ,np.percentile(Exposure,75)])

In [ ]:
Margin = (exposure_df.Lots * exposure_df.Price * LOT_SIZE * 0.3).to_list()

print("Average Margin Taken : {}".format(average(Margin)))
print("Minimum Margin : {}".format(min(Margin)))
print("Maximum Margin : {}".format(max(Margin)))
print("75th Percentile Margin : {}".format(np.percentile(Margin,75)))

rows.append(["Average Margin Taken :" ,average(Margin)])
rows.append(["Minimum Margin :" ,min(Margin)])
rows.append(["Maximum Margin :" ,max(Margin)])
rows.append(["75th Percentile Margin :" ,np.percentile(Margin,75)])

In [ ]:
Lots = exposure_df.Lots.to_list()

print("Average Lots traded : {}".format(int(average(Lots))))
print("Minimum Lots : {}".format(min(Lots)))
print("Maximum Lots : {}".format(max(Lots)))

In [ ]:
df.Date_Time = pd.to_datetime(df.Date_Time,format="%Y-%m-%d %H:%M:%S")

In [ ]:
# QUARTER WISE PNL
df['Year'] = (df.Date_Time.dt.year)
df['Quarter'] = ((df.Date_Time.dt.month - 1)/3).astype(int)
df['Six_month'] = ((df.Date_Time.dt.month - 1)/6).astype(int)

gp = df.groupby('Year')

Quarter = []
PNL = []

for k,res in gp:
    quart = res.groupby('Quarter')
    for q_num,qdata in quart : 
        Quarter.append('{} Quarter, {}'.format(q_num + 1, k))
        PNL.append(qdata.Balance.iloc[-1] - qdata.Balance.iloc[0])
        
quarter_df = pd.DataFrame()

quarter_df['Quarter'] = Quarter
quarter_df['PNL'] = PNL

# quarter_df.to_csv('V3_3_Quarter_PNL.csv')

In [ ]:
gp = df.groupby('Year')

Six = []
PNL = []

for k,res in gp:
    quart = res.groupby('Six_month')
    for q_num,qdata in quart : 
        Six.append('{} Half, {}'.format(q_num + 1, k))
        PNL.append(qdata.Balance.iloc[-1] - qdata.Balance.iloc[0])
        
six_df = pd.DataFrame()

six_df['Half'] = Six
six_df['PNL'] = PNL

# six_df.to_csv('V3_3_SIX_MONTHS.csv')


In [ ]:
exposure = np.percentile(Exposure,75)
profit_df = result_df[result_df['Result'] > 0]

In [ ]:
profit_df

In [ ]:
from numpy import short


long_trades = 0
short_trades = 0

for i in range(len(data['Balance'])):
    if(data.Result.iloc[i] != 0):
        if(data.Strategy.iloc[i] == "Long Strategy"):
            long_trades = long_trades + 1
        else:
            short_trades = short_trades + 1
        

ro1 = ['TOTAL NUMBER OF TRADES : ',str(long_trades + short_trades)]        
print("TOTAL NUMBER OF TRADES : " + str(long_trades + short_trades))
ro2 = ['LONG STRATEGY TRADES : ',str(long_trades)] 
print("LONG STRATEGY TRADES : " + str(long_trades))
ro3 = ['SHORT STRATEGY TRADES : ',str(short_trades)] 
print("SHORT STRATEGY TRADES : " + str(short_trades))

rows.append(ro1)
rows.append(ro2)
rows.append(ro3)


In [ ]:
data['Date_Time'] = pd.to_datetime(data['Date_Time'],format='%Y-%m-%d %H:%M:%S')

data['Date'] = data.apply(lambda x: x.Date_Time.date(),axis = 1)
data['Month'] = data.apply(lambda x: x.Date.month,axis = 1)

print(data)

In [ ]:
#Best and worst trades
profit = 0
profit_i = 0
loss = 100
loss_i = 0 
for i in range(len(data['Balance'])):
    if(data.Result.iloc[i] > profit):
        profit = data.Result.iloc[i]
        profit_i = data.Date_Time.iloc[i]
    if(data.Result.iloc[i] < loss):
        loss = data.Result.iloc[i]
        loss_i = data.Date_Time.iloc[i]

print("MAX PROFIT " + str(profit) + " at " + str(profit_i))
print("MAX LOSS " + str(loss) + " at " + str(loss_i))

ro1 = ['Max Profit : ', str(profit),'Time : ', str(profit_i)]
ro2 = ['Max Loss : ', str(loss),'Time : ', str(loss_i)]

rows.append(ro1)
rows.append(ro2)

In [ ]:
# Finding monthly returns 

returns_monthly = [] 
balance_monthly =[]
balance_tradely = []
returns_tradely = []

positive_returns = []
negative_returns = []

dt_monthly = []
dt_tradely = []
prev_month = None

balance_tradely.append(10000000)
dt_tradely.append(data.Date_Time.iloc[0])

balance_monthly.append(10000000)
dt_monthly.append(data.Date.iloc[0])
for i in range(len(data['Date_Time'])):
    if((data.Result.iloc[i] != 0)):
        balance_tradely.append(data.Balance.iloc[i])
        dt_tradely.append(data.Date_Time.iloc[i])
        cur_month = data.Month.iloc[i]
        if(prev_month != None and cur_month != prev_month):
            balance_monthly.append(data.Balance.iloc[i-1])
            dt_monthly.append(data.Date.iloc[i])
        if(i == len(data['Date_Time'])-1):
            balance_monthly.append(data.Balance.iloc[i])
            dt_monthly.append(data.Date.iloc[i])
        prev_month = cur_month    

# print(len(balance_monthly))

for i in range(1,len(balance_monthly)):
    cur = (100*(balance_monthly[i] - balance_monthly[i-1]))/(exposure)
    returns_monthly.append(cur)
    
for i in range(1,len(balance_tradely)):
    cur = (100*(balance_tradely[i] - balance_tradely[i-1]))/(exposure)
    if(cur >= 0):
        positive_returns.append(cur)
    else:
        negative_returns.append(cur)
    returns_tradely.append(cur)

In [ ]:
# ANNUALIZED RESULTS 
balance_yearly = []
returns_yearly = []
balance_yearly.append(10000000)

for i in range(len(data['Date_Time'])):
    cur_month = data.Month.iloc[i]
    nxt_month = None
    if(i+1 < len(data['Date_Time'])):
        nxt_month = data.Month.iloc[i+1]
        
    if(cur_month != 4 and nxt_month == 4):
        balance_yearly.append(data.Balance.iloc[i])
        
    # if(i == len(data['Date_Time']) - 1):
    #     balance_yearly.append(data.Balance.iloc[i])

balance_yearly.append(data.Balance.iloc[len(data['Date_Time']) - 1])
     
# print(balance_yearly)
for i in range(1,len(balance_yearly)):
    cur = (100*(balance_yearly[i] - balance_yearly[i-1]) )/(exposure)
    returns_yearly.append(cur)


In [ ]:
print(returns_yearly)
# print(balance_monthly)

In [ ]:
balance_mon = pd.DataFrame()

balance_mon['Month_Start'] = dt_monthly
balance_mon['Portfolio_Value'] = balance_monthly

balance_mon.reset_index()
balance_mon.set_index('Month_Start')

balance_mon.to_csv(MONTHLY_CSV)

In [ ]:

max_drawdown = 0

i = 0 

start_drawdown = -1
end_drawdown = -1

while(i < len(balance_tradely)):
    j = i + 1
    while(j < len(balance_tradely)):
        cur_drawdown = balance_tradely[i] - balance_tradely[j]
        if(cur_drawdown>max_drawdown):
            max_drawdown = cur_drawdown
            start_drawdown = i
            end_drawdown = j
        j = j + 1
    i = i + 1

print("MAX DRAWDOWN AMOUNT: " + str(max_drawdown))
print("MAX DRAWDOWN % : " +str(100*(max_drawdown/exposure)))
print(start_drawdown)
print(end_drawdown)

print('START : {}'.format(dt_tradely[start_drawdown]))
print('END : {}'.format(dt_tradely[end_drawdown]))

ro1 = ['Max Drawdown Amount : ' , str(max_drawdown)]
rows.append(ro1)

ro2 = ['Max Drawdown % : ' , str(100*(max_drawdown/exposure))]
rows.append(ro2)

In [ ]:
balance_tradely

In [ ]:
from numpy import average, std


sharpe_ratio = 0

avg_returns = average(returns_monthly)
sigma = std(returns_monthly)

sharpe_ratio = round((avg_returns) / sigma,2 )

print(sharpe_ratio)

In [ ]:
from numpy import average


avg_return_per_trade = round(average(returns_tradely),2)
avg_return_per_month = round(average(returns_monthly),2)
avg_yearly_return = round(average(returns_yearly),2)
avg_positive_return = round(average(positive_returns),2)
avg_negative_return = round(average(negative_returns),2)


print("AVERAGE RETURN PER TRADE : " + str(avg_return_per_trade))
print("AVERAGE RETURN PER MONTH : " + str(avg_return_per_month))
print("AVERAGE RETURN PER YEAR : " + str(avg_yearly_return))
print("POSIIVE AVERAGE RETURN ON TRADES: " + str(avg_positive_return))
print("NEGATIVE AVERAGE RETURN ON TRADES: " + str(avg_negative_return))
print("NUMBER OF POSITIVE TRADES : " + str(len(positive_returns)))
print("NUMBER OF NEGATIVE TRADES : " + str(len(negative_returns)))
rows.append(["AVERAGE RETURN PER TRADE : " , str(avg_return_per_trade)])
rows.append(["AVERAGE RETURN PER MONTH : " , str(avg_return_per_month)])
rows.append(["AVERAGE RETURN PER YEAR : " , str(avg_yearly_return)])
rows.append(["POSIIVE AVERAGE RETURN ON TRADES: " , str(avg_positive_return)])
rows.append(["NEGATIVE AVERAGE RETURN ON TRADES: " , str(avg_negative_return)])
rows.append(["NUMBER OF POSITIVE TRADES : " , str(len(positive_returns))])
rows.append(["NUMBER OF NEGATIVE TRADES : " , str(len(negative_returns))])

In [ ]:
#AVG HOLDING PERIOD

from cmath import isnan


avg_hold = 0 
cnt = 0
start = None
end = None
start = pd.to_datetime(df.Date_Time.iloc[0])
for i in range(len(df['Balance'])):
    if(df.Result.iloc[i] == 0):
        None
    else:
        end = pd.to_datetime(df.Date_Time.iloc[i])
        diff = (end - start)
        if(cnt == 0):
            avg_hold = diff
        else:
            avg_hold = avg_hold + diff
        cnt = cnt+1
        # print(diff)
        end = None
        if(i+1 < len(df['Balance'])):
            start = pd.to_datetime(df.Date_Time.iloc[i+1])

avg_hold = avg_hold / cnt

hrs = int(avg_hold.total_seconds() / 3600)
mins = int(avg_hold.total_seconds() / 60) - 60 * hrs
secs = int(avg_hold.total_seconds()) - 60*mins - 3600*hrs

print("Average Holding Period : " + str(hrs) + ":" +str(mins) + ":" + str(secs))
print("Average Holding Period : " + str(avg_hold))
       
rows.append(["Average Holding Period : " , str(hrs) + ":" +str(mins) + ":" + str(secs)])
rows.append(["Average Holding Period : " , str(avg_hold)])

In [ ]:
# Number of trades excluding rollovers 

num_trades = 0

for i in range(len(df['Balance'])):
    if(df.Strategy.iloc[i] != 'Rollover'):
        num_trades = num_trades + 1
    else:
        None
print("Number of trades without rollovers : " + str(num_trades))

rows.append(["Number of trades without rollovers : " , str(num_trades)])

In [ ]:
# To be printed 

rows.append(['SHARPE RATIO : ' , str(sharpe_ratio)])

with open(STATS_CSV, 'w') as f:
    writer = csv.writer(f)
    
    for i in rows:
        writer.writerow(i)

In [ ]:
from datetime import datetime
BALANCE = []
past = 10000000
i = 0

# df_x = pd.read_csv('CRUDEOIL-1.csv')

df_x['Date_Time'] = pd.to_datetime(df_x['Date'] + df_x['Time'], format='%Y-%m-%d%H:%M:%S')

df_x.Date_Time = pd.to_datetime(df_x.Date_Time)

# df_x = df_x[df_x.Date_Time >= datetime(2022,4,18,1,1,1)]
# df_x = df_x[df_x.Date_Time >= datetime(2015,1,1,1,1,1)]

for n in range(len(df_x['Open'])):
    if(i == len(data['Balance'])):
        BALANCE.append(past)
    else:
        cur_date = df_x.Date_Time.iloc[n].date()
        upd_date = pd.to_datetime(data.Date_Time.iloc[i]).date()
        if(cur_date == upd_date):
            past = data.Balance.iloc[i]
            i = i + 1
        BALANCE.append(past)

df_x['Balance'] = BALANCE
    
        

In [ ]:

from datetime import date,datetime
from tkinter import E
from turtle import color
from matplotlib.font_manager import font_scalings
import matplotlib.pyplot as plt
import matplotlib

plt.rcParams["figure.figsize"] = (20,15)


plt.title('EQUITY_CURVE')

# data.Date_Time = pd.to_datetime(data.Date_Time)

x1 = pd.to_datetime(data['Date_Time'],format='%d-%m-%Y %H:%M')
y1 = data['Balance']

x2 = pd.to_datetime((df_x['Date_Time']))
y2 = df_x['Close']

ax1 = plt.subplot(211)
plt.plot(x1,y1)

plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

plt.grid()

ax2 = plt.subplot(212,sharex=ax1)
plt.plot(x2,y2)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.grid()
# font = {'family' : 'normal',
#         'weight' : 'bold',
#         'size'   : 22}

# matplotlib.rc('font', **font)
# plt.plot(x1,y1)
# plt.xlabel("Time")
# plt.ylabel("Equity Value")

# plt.title("Equity Curve")

# plt.savefig(EQUITY_CURVE)

ax1.set_title('EQUITY_CURVE')
ax2.set_title('UNDERLYING')

plt.savefig(EQUITY_CURVE_2)
plt.legend()
plt.show()

In [ ]:
from datetime import datetime

year_start = ['01-04-2013 09:01','01-04-2014 09:01','01-04-2015 09:01','01-04-2016 09:01','01-04-2017 09:01','01-04-2018 09:01','01-04-2019 09:01','01-04-2020 09:01','01-04-2021 09:01','01-04-2022 09:01']

cnt = 0

for i in range(len(year_start)):
    if(cnt == 10):
        break
    cnt = cnt + 1
    tmp_1 = data
    start_date = datetime.strptime(year_start[i],'%d-%m-%Y %H:%M')
    end_date = datetime.strptime(year_start[i+1],'%d-%m-%Y %H:%M')
    tmp_1 = tmp_1.where(tmp_1['Date_Time'] >= start_date)
    tmp_1 = tmp_1.where(tmp_1['Date_Time'] < end_date)
    tmp_1.dropna(axis=0,inplace = True)
    x1 = tmp_1['Date_Time']
    y1 = tmp_1['Balance']
    
    tmp_2 = df_x
    tmp_2 = tmp_2.where(tmp_2['Date_Time'] >= start_date)
    tmp_2 = tmp_2.where(tmp_2['Date_Time'] < end_date)
    tmp_2.dropna(axis=0,inplace = True)
    
    # x1 = tmp_2['Date_Time']
    # y1 = tmp_2['Balance']
        
    x2 = tmp_2['Date_Time']
    y2 = tmp_2['Close']
    
    ax1 = plt.subplot(211)
    plt.plot(x1,y1)

    ax2 = plt.subplot(212,sharex=ax1)
    plt.plot(x2,y2)
    
    # plt.plot(x1,y1)
    # plt.xlabel('Time')
    # plt.ylabel('Equity Value')

    # plt.title(datetime.strptime(year_start[i],'%d-%m-%Y %H:%M').year,loc='left')
    yr = datetime.strptime(year_start[i],'%d-%m-%Y %H:%M').year
    
    # plt.title('Year ' + str(yr))
    
    plt.savefig(YEAR_PLOTS + '\\' + str(yr)+'_{}.jpg'.format(DAYS))
    
    plt.show()
    # break

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
%matplotlib inline
plt.rcParams.update({'figure.figsize':(7,5), 'figure.dpi':100})

# Plot Histogram on x
x = returns_tradely
plt.hist(x, bins=100,rwidth=0.5)
# plt.xlim(50,75)
# font = {'family' : 'normal',
#         plt.axis,
#         'size'   : 1}
plt.xticks(rotation=90)
# matplotlib.rc('font', **font)
plt.xticks(np.arange(min(returns_tradely), max(returns_tradely)+1, 5.0))
# plt.xticks(100)
plt.gca().set(title='Frequency Histogram', ylabel='Frequency')

In [ ]:
df1['TMP_SD'] = ((df1['Theoritical H10'] - df1['Theoritical L10'])*100)/(df1['Theoritical H10'])

print(max(df1.TMP_SD))
print(max(df1.SD_3_15))

In [ ]:
trade(df1,df2)